Creating a balanced test set for flat training (cross validation)

In [1]:
import os
import yaml
import pickle

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
cd ../../src/

/Users/cock/kDrive/PhD/Projects/Labs/beerslaw-lab/src


# Data


In [3]:
# experiment
with open('../data/experiment_keys/over10.pkl', 'rb') as fp:
    over10 = pickle.load(fp)

# data
with open('../data/post_test/rankings_scored.pkl', 'rb') as fp:
    rankings = pickle.load(fp)
    rankings = rankings[rankings['username'].isin(over10)]
    
# vector map
map_path = '../data/experiment_keys/permutation_maps/vector_binary.yaml'
with open(map_path) as fp:
    vector_map = yaml.load(fp, Loader=yaml.FullLoader)

In [4]:
rankings['prior_4cat_knowledge'] = rankings['prior_4cat'].apply(lambda x: x[0] if x != 'none' else 0)
rankings['total_score'] = rankings['q1_score'] + rankings['q2_score'] + rankings['q3_score'] + rankings['q4_score'] + rankings['q5_score'] + rankings['q6_score']
rankings['vector_binary'] = rankings['ranking'].apply(lambda x: vector_map['map'][x])
rankings['vector_score'] = rankings['vector_binary'].apply(lambda x: int(str(x)[0]) + int(str(x)[1]) + int(str(x)[2]))
rankings['vector_binconcepts'] = rankings['vector_score'].apply(lambda x: int(x >= 2))

In [5]:
# stratification columns (ideal separation)
stratification = [
    'ranking', 'year', 'gender', 'prior_4cat_knowledge', 'total_score', 'vector_binary',
    'vector_score', 'vector_binconcepts'
]


In [6]:
train, test = train_test_split(rankings, test_size=0.20, random_state=0, stratify=rankings[['ranking']])

In [7]:
for strat in stratification:
    
    test_strat = test[[strat, 'username']].groupby(strat).nunique().reset_index()
    test_strat['username'] = test_strat['username'] / sum(test_strat['username'])
    test_strat.columns = [strat, 'test']
    
    train_strat = train[[strat, 'username']].groupby(strat).nunique().reset_index()
    train_strat['username'] = train_strat['username'] / sum(train_strat['username'])
    train_strat.columns = [strat, 'train']
    
    strat_df = test_strat.merge(train_strat, on=strat, how='inner')
    
    print(strat)
    display(strat_df)
    print()

ranking


,ranking,test,train
0,0123,0.019231,0.029412
1,0213,0.076923,0.083333
2,0231,0.115385,0.112745
3,0312,0.038462,0.039216
4,0321,0.019231,0.024510
5,1023,0.019231,0.009804
6,1032,0.019231,0.009804
7,1203,0.019231,0.009804
8,1320,0.019231,0.019608
9,2013,0.076923,0.063725



year


,year,test,train
0,1st,0.384615,0.377451
1,2nd,0.365385,0.446078
2,3rd,0.250000,0.176471



gender


...

,vector_binary,test,train
0,000,0.346154,0.338235
1,001,0.096154,0.102941
2,010,0.038462,0.044118
3,011,0.134615,0.142157
4,100,0.076923,0.083333
5,101,0.115385,0.112745
6,110,0.076923,0.063725
7,111,0.115385,0.112745



vector_score


,vector_score,test,train
0,0,0.346154,0.338235
1,1,0.211538,0.230392
2,2,0.326923,0.318627
3,3,0.115385,0.112745



vector_binconcepts


,vector_binconcepts,test,train
0,0,0.557692,0.568627
1,1,0.442308,0.431373


In [8]:
with open('../data/experiment_keys/flatstrat_testusernames.pkl', 'wb') as fp:
    pickle.dump(list(test['username']), fp)
with open('../data/experiment_keys/flatstrat_trainusernames.pkl', 'wb') as fp:
    pickle.dump(list(train['username']), fp)

In [9]:
print('There are {} train instances, and {} test instances'.format(len(train), len(test)))

There are 204 train instances, and 52 test instances
